# Implicit Solvent Sample-Based Quantum Diagonalization

*Usage estimate: 10 seconds on ibm_torino (NOTE: This is an estimate only. Your runtime might vary.)*

Challenge Based on [*Implicit Solvent Sample-Based Quantum Diagonalization* - J. Phys. Chem. B 2023, 127 (21), 4471–4482.](https://pubs.acs.org/doi/10.1021/acs.jpcb.5c01030)

The recent work, $\textit{Implicit Solvent Sample-Based Quantum Diagonalization}$, directly addressed the question: “Can Sample-Based Quantum Diagonalization energy calculations be reliably integrated with established classical solvation models to reproduce the outcomes of classical energy computations for the simplest biologically relevant polar molecules?” Their findings demonstrate that the answer is a resounding yes.

But a pressing question yet to be answered within scientific literature is "Within the solute-solvent computational context, can SQD add value beyond the established classical routines?" 

This notebook provides a scaling roadmap for SQD-based Quantum Chemistry experiments implemented on real quantum hardware from small-scale, classically simulable cases to large-scale, utility grade demonstrations. By the successful completion of all stages, you will be at the liminal boundary of State-of-the-Art, Utility-Scale Quantum Chemistry experimentation on Quantum Computers.



<div class="alert alert-block alert-success">

## **Contents of this Notebook**:

## **Challenge**: Methylamine, $CH_{3}NH_{2}$ 

### Participants must implement code in several sections to score points

 1. **Hartree-Fock Implementation** : (local execution)

    Changes needed in this notebook `.\ch2_implicit_solvent.ipynb`

 2. **Classical implementation**    :  (remote execution)  : Qiskit Serverless

    Changes needed in this notebook `.\ch2_implicit_solvent.ipynb`

    Changes needed in `.\source_files\classical_simulation.py`

 3. **Quantum implementation**      :  (remote execution)  : Qiskit Runtime, Qiskit Serverless
 
    Changes needed in this notebook `.\ch2_implicit_solvent.ipynb`

You should draw from the provided reference [Implicit Solvent Sample-Based Quantum Diagonalization](https://pubs.acs.org/doi/10.1021/acs.jpcb.5c01030) to guide your selections in this exploration.

</div>

<div class="alert alert-block alert-warning">

## **Challenge Rules**
1.  You need to save a final plotted image using the provided function in order for your team's score to be considered valid.
2.  You need to save your results using the `results.json` file output by the provided `report_results()` function in order for your team's score to be considered valid.

</div>

<div class="alert alert-block alert-success">

# Challenge instructions: Molecule of Methylamine

### Your tasks

Implement code to perform a small-scale example that demonstrates the CASCI/IEF-PCM(cc-pVDZ) vs SQD/IEF-PCM(cc-pVDZ) workflow on acqueous Methylamine to compute solute-solvent interactions by following directions and filling in code below. Look for the `# PROMPT` marker inside the code boxes to find the places where code needs to be filled in. Where needed, there are specific `# BEGIN ANSWER` and `# END ANSWER` markers as well. Completing the notebook by filling in the prompts will lead to a basic implementation that has a few simplifications relative to the implementation in [Implicit Solvent Sample-Based Quantum Diagonalization](https://pubs.acs.org/doi/pdf/10.1021/acs.jpcb.5c01030?ref=article_openPDF).

## Grading system (100 points):
### Ex.1: Obtaining the Restricted Hartree Fock energy (20 points) 
### Ex.2: Obtaining the correct CASCI classical baseline (30 points) 
### Ex.3: Obtaining the correct SQD-based value (up to 50 points)
<div>


## Python imports

We assume that we start from a Python environment that has been initialized by following the instructions.

In [ ]:
import pyscf
from pyscf import gto   # Deals with molecular initialization
from pyscf import scf   # Solvation methods
from pyscf import mcscf # More computational solvation methods
from pyscf.solvent import pcm
from qiskit_ibm_catalog import QiskitFunction, QiskitServerless
from qiskit_ibm_runtime import QiskitRuntimeService

Import the grader functions and submit the team name to which your points should be assigned:

In [ ]:
from grader_ch2 import (challenge2, grader_ch2_ex1, grader_ch2_ex2, grader_ch2_ex3 )
# PROMPT Define your team name
YOUR_TEAM_NAME = "r2p_converters"
challenge2.submit_name(YOUR_TEAM_NAME)

### Initialize your results file

In [ ]:
# Call this cell to initialize your results file. Be careful not to call this when you don't want to! Calling it several times will overwrite your results!
from utils.helpers import init_results_file

while True:
    print("Do you want to initialize a new results file? (y/n)")
    answer = input()
    if answer == "y":
        init_results_file()
        break
    elif answer == 'n':
        break

<div class="alert alert-block alert-warning">

## **Excercise 1**: Methylamine - Restricted Hartree-Fock
20 Points

</div>

### Initialize the molecule object by using known $\textit{a priori}$ molecular geometry

In [ ]:
# Retrieve molecular geometries from:
# - option 1: https://pubchem.ncbi.nlm.nih.gov
# - option 2: somewhere else of your choosing
 

molecule_name = 'Methylamine'
# PROMPT: Define the geometry
methylamine_geo = """
    N   <XXX>    <XXX>    <XXX>;
    C   <XXX>    <XXX>    <XXX>;
    H   <XXX>    <XXX>    <XXX>;
    H   <XXX>    <XXX>    <XXX>;
    H   <XXX>    <XXX>    <XXX>;
    H   <XXX>    <XXX>    <XXX>;
    H   <XXX>    <XXX>    <XXX>;
"""

In [ ]:
# Explicitly defining the Methylamine molecule
mol = gto.M()
mol.atom = methylamine_geo
# PROMPT: Define the properties
# BEGIN ANSWER
mol.basis = 
mol.unit =
mol.charge =
mol.spin =
mol.verbose =
# END ANSWER

mol.build()

### Define solvation effects by using the polarizable continuum model (PCM)

In [ ]:
# Retrieve dielectric parameters from: https://gaussian.com/scrf/

# PROMPT:
eps_water = 

In [ ]:
cm = pcm.PCM(mol)
cm.eps = eps_water
# PROMPT:
cm.method = 

In [ ]:
from pyscf import scf   # Self-Consistent Field methods

# Creating a "Restricted Hartree Fock" object for the solute, then wrapping the SCF object with a Polarizable Continuum Model
mf_pcm0 = scf.RHF(mol).PCM(cm) # Restricted Hartree Fock misses instantaneous correlations, post-HF methods like CCSD, CI, MP2 might be worth exploring

### Geometry optimization by using TRIC

In [ ]:
# Geometry optimization with geomeTRIC
from pyscf.geomopt.geometric_solver import optimize # GeomeTRIC under the hood, for geometry optimization
mol_opt = optimize(mf_pcm0, tol_grad=3e-4, verbose=0) # Use geomeTRIC/TRIC under the hood

### Compute the restricted Hartree-Fock energy


In [ ]:
# Run the kernel to get the RHF energy
mf_opt = scf.RHF(mol_opt).PCM(cm)
hf_e = float(mf_opt.kernel())

In [ ]:
print(f"Restricted Hartree-Fock Energy: {hf_e}")

<div class="alert alert-block alert-danger">

## Record your restricted Hartree-Fock results

To report your results, use the function $report\_result()$. You can put your finalized classical energy and quantum energy values for each respective molecule. Every time you finish a molecule, make sure to put your answers into the `report_result` function, which will write them to a `results.json` file that will be used to compute your score. 

</div>

In [ ]:
from utils.helpers import report_result

# Overwrite current data in results.json (be careful, and save a backup file if uncertain)
report_result(molecule_name=molecule_name, hartree_fock_E=hf_e, casci_E=None, sqd_E=None)

In [ ]:
# Check your grade
grader_ch2_ex1(molecule_name=molecule_name, hartree_fock_E=hf_e)

<div class="alert alert-block alert-success">
Good job! You've finished Excercise 1!
</div>

<div class="alert alert-block alert-warning">

## **Excercise 2**: Methylamine CASCI/(cc-pVDZ) IEF-PCM (by using Qiskit Serverless)
30 Points

</div>

### **Qiskit Serverless**:

Qiskit Serverless is a framework for running distributed quantum and classical workloads without managing infrastructure. There is no server provisioning (no spinning up EC2s, clusters, Docker containers), no orchestration tools (Kubernetes, Docker Swarm), no monitoring/maintenance, etc. Each job run through serverless runs in a clean container, executes your code, and then shuts down. There is no memory between jobs. You just write your code, define dependencies, and submit your job.

Qiskit Serverless gives a user access to always-on remote CPU cores and memory, allowing certain classical workloads to be distributed across the remote resources and to gain some advantage in parallel program processing while avoiding common headaches from device shutdown mid-execution.

In summary, Qiskit Serverless has these benefits:

1) Is it an always-on remote environment.
2) It has mltiple remote CPU cores, allowing us to distribute computation efficiently across parallel resources.

In [ ]:
# Establish Qiskit Runtime connection
client = QiskitServerless(name="r2p-2025")

<div class="alert alert-block alert-warning">

Visit `./source_files/classical_simulation.py` and fill in the code (based on Excercise 1) to complete this section.

</div>

In [ ]:
# We need to share the program intended to run in the cloud environment, and re-upload it any time we change it's source code
client.upload(
    QiskitFunction(
        title="classical_simulation",
        entrypoint="classical_simulation.py", # lives in ./source_files
        working_dir="source_files"))

In [ ]:
# Setup a Serverless Client
worker = client.load("classical_simulation")

In [ ]:
from json.encoder import JSONEncoder
# PROMPT: Get the labels from Table 1 of the paper
ao_labels = ['', '', '', '', '']
data_e = JSONEncoder().encode([mol_opt.tostring(), eps_water, ao_labels])

In [ ]:
serverless_job = worker.run(data=data_e)

In [ ]:
from utils.helpers import feedback_serverless
# Optionally, check the Serverless status feedback
# Don't sit here and stare at the feedback unless debugging. You can go develop something else while the Serverless job runs.
_ = feedback_serverless(serverless_job)

In [ ]:
#If you make a mistake and need to cancel something
# for job in client.jobs():
#     job.cancel()

In [ ]:
from json.decoder import JSONDecoder
CASCI_E = JSONDecoder().decode(serverless_job.result()['outputs'])[0]

In [ ]:
# We have approximated the red, classical baseline from Figure 1 for Methanol (North-West panel)
print(f"CASCI/IEF-PCM(cc-pVDZ): E={CASCI_E}")

<div class="alert alert-block alert-danger">

### Record your CASCI results

To report your results, use the $report\_result()$ function. You can put your finalized classical energy and quantum energy values for each respective molecule. Every time you finish a molecule, make sure to put your answers into the `report_result` function, which will write them to a `results.json` file that will be used to compute your score. 

</div>

In [ ]:
from utils.helpers import report_result

# Overwrite current data in results.json (be careful, and save a backup file if uncertain)
# This is used for plotting at the end
report_result(molecule_name=molecule_name, hartree_fock_E=hf_e, casci_E=CASCI_E, sqd_E=None)

In [ ]:
# Check your grade
grader_ch2_ex2(molecule_name, CASCI_E)

<div class="alert alert-block alert-success">
Good job! You've finished Excercise 2!
</div>

<div class="alert alert-block alert-warning">

## **Excercise 3**: Methylamine SQD/(cc-pVDZ) IEF-PCM (by using Qiskit Serverless)
50 Points

</div>

### Establish a Qiskit Runtime connection

In [ ]:
# Establish Quantum Resource connection

service = QiskitRuntimeService(name="r2p-2025")
# PROMPT: Choose your prefered backend
backend_name = 
backend = service.backend(backend_name, use_fractional_gates=True)
print(f"Using backend {backend.name}")

### Establish a Qiskit Serverless connection (if not done for the classical baseline calculation)

In [ ]:
# Establish Classical HPC Resource connection

client = QiskitServerless(name="r2p-2025" )

In [ ]:
from pyscf.mcscf import avas

# Re-define the PCM
cm = pcm.PCM(mol_opt)
cm.eps = eps_water # for water
# PROMPT: you can use the same method as before
cm.method =

# Re-build the Restricted Hartree Fock object
mf_opt = scf.RHF(mol_opt).PCM(cm)
mf_opt.kernel(verbose=0)

# Run AVAS
# PROMPT: use the same ao_levels as before
ao_labels = [, , , , ]
avas_ = avas.AVAS(mf_opt, ao_labels, with_iao=True, canonicalize=True, verbose=0)
avas_.kernel()
norb, ne_act, mo_avas = avas_.ncas, avas_.nelecas, avas_.mo_coeff

num_elec_a = (ne_act + mol_opt.spin) // 2
num_elec_b = (ne_act - mol_opt.spin) // 2

In [ ]:
from qiskit.transpiler import generate_preset_pass_manager
import ffsim

from utils.sqd_helpers import get_zigzag_physical_layout

# Initial LUCJ ansatz layout
initial_layout, _ = get_zigzag_physical_layout(norb, backend=backend)

# Initialize a pass manager 
# PROMPT: define your basis gates, e.g. ['rx', 'rz', 'cz']
pass_manager = generate_preset_pass_manager(
    optimization_level=3, backend=backend, initial_layout=initial_layout, basis_gates=[]
)

pass_manager.pre_init = ffsim.qiskit.PRE_INIT

In [ ]:
# We need to share the program intended to run in the cloud environment, and re-upload it any time we change it's source code
client.upload(
    QiskitFunction(
        title="diagonalization_engine",
        entrypoint="diagonalization_engine.py", # lives in ./source_files
        working_dir="source_files"))

In [ ]:
# Sanity Check a Representative Layout
from pyscf import ao2mo
from utils.heartwood import run_active_space_calculation, get_lucj
from utils.visualization import used_qubits, color_batches
from utils.gate_map import plot_gate_map

mc = pyscf.mcscf.CASCI(mf_opt, ncas=norb, nelecas=ne_act).PCM(cm)
mc.with_solvent.method = mf_opt.with_solvent.method  #  Here was make sure that mc is also using the same solvent method defined earlier (IEF-PCM)
mc.with_solvent.eps = mf_opt.with_solvent.eps # Set the dielectric paramters
mc.mo_coeff = mo_avas.copy() # Update the molecular orbitals to include those computed in the presence of the solvent

h1e_cas, ecore = mc.get_h1eff() # <-- h1eff is the 1-electron hamiltonian integrals. h1e_cas is a common alias. ecore is the electronic energy from the core orbitals.
h2e_cas = ao2mo.restore(1, mc.get_h2eff(), norb) # <-- get the 2-electron hamiltonian integrals

mc.mo_coeff, t1, t2 = run_active_space_calculation(h1e_cas, h2e_cas, norb, ne_act, mo_avas.copy(), mf_opt.mol.nelectron//2, ecore)
circuit = get_lucj(norb, num_elec_a, num_elec_b, t1, t2, n_reps=1)
isa_circuit = pass_manager.run(circuit)

qubits = used_qubits(isa_circuit, include={"unitary"})
qcolors = color_batches([set(qubits), set(qubits)^set(range(0, backend.configuration().n_qubits))])
plot_gate_map(backend, label_qubits=True, qubit_color=qcolors, line_color=["black" for _ in backend.coupling_map.get_edges()])

### User controls 
Systematically vary these parameters to improve hardware results.


In [ ]:
# Set to "True" to run on real QPU hardware
# Set to "False" to sample from a uniform distribution on classical hardware (useful for parameter debugging)
# If you do not submit results from real Quantum Hardware, you will receive zero points for the section!
# PROMPT:
use_hardware = 

# Error Suppression/Mitigation Options
# >> Configure within at Sampler Primitive

# Transpiler Options
# PROMPT:
optimization_level = 

# Heartwood Algorithm Options
# PROMPT:
n_iter =   # How many update loops to run
resample =  # (resample=1 -> resample the QPU after every update loop; resample=n_iter -> sample QPU only once)
shots =  # <-- Get reasonable shots from Figure 5 (REF: Implicit Solvent Sample-Based Quantum Diagonalization)

# SQD Options
# PROMPT:
energy_tol = 
occupancies_tol = 
max_iterations = 

# Eigenstate Solver Options
# PROMPT:
num_batches = 
samples_per_batch = 
carryover_threshold = 
symmetrize_spin = True
max_cycle = 

# Classical Post-Processing Options
# PROMPT:
mem =  # Memory allocated to each parallel Serverless worker (Gb)

In [ ]:
# The Heartwood Algorithm
import time
import numpy as np

from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_addon_sqd.counts import generate_bit_array_uniform
from pyscf import ao2mo

from utils.heartwood import run_active_space_calculation, update_rdm, get_lucj, classically_diagonalize

# TIP: Decide if running the hardware jobs in the Session qiskit_ibm_runtime primitive will help, 
# given the time constraint vs parameter exploration tradeoff that you must navigate in the challenge

# Sampler Primitive Options 
sampler = Sampler(mode=backend)
sampler.options.max_execution_time = 30 # Don't change this unless you are confident that you should

# Explore error suppression techniques
# sampler.options.dynamical_decoupling.enable = '<XXX>'
# sampler.options.dynamical_decoupling.sequence_type = '<XXX>'
# sampler.options.twirling.enable_measure = '<XXX>'
# sampler.options.twirling.enable_gates = '<XXX>'
# sampler.options.twirling.num_randomizations = '<XXX>'
# sampler.options.twirling.shots_per_randomization = '<XXX>'

# initial approximation for rdm1
with_solvent_e, with_solvent_v = None, None # Don't touch
data = []
for iiter in range(n_iter):
    print(f">>>>> IMPLICIT SOLVENT ITERATION {iiter+1}/{n_iter}")
    if with_solvent_v is not None: 
        # Subsequent update loops enter here
        mc.get_hcore = lambda *args: mc._scf.get_hcore() + with_solvent_v
    else: 
        # First update loop starts here
        # hcore is the CAS space (classically computed) 1-electron hamiltonian, which we default to at the start of the routine.
        mc.get_hcore = lambda *args: mc._scf.get_hcore() # REF: https://pyscf.org/pyscf_api_docs/pyscf.mcscf.html#pyscf.scf.hf.CASBase.get_h1cas

    # Alias mapping
    # hcore : h1e_cas : h1e_eff
    # nuclear_repulsion_energy : ecore
    # eri : h2e_cas : h2e_eff

    h1e_cas, ecore = mc.get_h1eff() # <-- h1eff is the 1-electron hamiltonian integrals. h1e_cas is a common alias. ecore is the electronic energy from the core orbitals.
    h2e_cas = ao2mo.restore(1, mc.get_h2eff(), norb) # <-- get the 2-electron hamiltonian integrals

    mc.mo_coeff, t1, t2 = run_active_space_calculation(h1e_cas, 
                                                        h2e_cas, 
                                                        norb, 
                                                        ne_act, 
                                                        mo_avas.copy(), 
                                                        mf_opt.mol.nelectron//2, 
                                                        ecore)

    if use_hardware == True:
        if iiter % resample == 0: # <-- Toggle how often you refresh your bitstrings here. The developer suggests that you do it every time, but benevolently provides the freedom to disagree with him via the resample control variable.
            # The "Quantum-Centric" part
            print(">>>>> GENERATING BITSTRINGS USING QUANTUM HARDWARE")
            # LUCJ Ansatz construction
            circuit = get_lucj(norb, num_elec_a, num_elec_b, t1, t2, n_reps=1)

            print(f">>>>> TRANSPILING LUCJ TO {backend.name}")
            isa_circuit = pass_manager.run(circuit)
            print(f">>>>> SUBMITTING ISA_CIRCUIT TO {backend.name}")
            job = sampler.run([isa_circuit], shots=shots) # <----- Error Suppression/Mitigation configured performed upstream
            job_id = str(job.job_id())

            timer = 0
            while job.status() != "DONE":
                timer+=10
                print(f">>>>> [{timer}s] RUNTIME JOB {job_id}: {job.status()}")
                time.sleep(10)

            primitive_result = job.result()
            print(f">>>>> RETRIEVED {job_id} FROM {backend.name}")

            pub_result = primitive_result[0]
            bit_array = pub_result.data.meas

    else:
        print(">>>>> GENERATING BITSTRINGS CLASSICALLY")
        rng = np.random.default_rng(24)
        bit_array = generate_bit_array_uniform(100_000, 2*norb, rand_seed=rng) # <-- Sample bitstrings from a uniform distribution. This is useful for debug, but runs out of steam on large-systems
        job_id = float("nan") # <-- we will check that valid job_id's were passed during grading

    # The "Classical Post-processing" part
    result = classically_diagonalize(bit_array=bit_array, 
                                    nuclear_repulsion_energy=ecore,           # Electronic energy from the core orbitals
                                    hcore=h1e_cas,                            # 1-electron hamiltonian integrals
                                    eri=h2e_cas,                              # 2-electron hamiltonian integrals
                                    num_orbitals=norb,                        # Number of spatial orbitals
                                    nelec=ne_act,                             # Number of electrons
                                    num_elec_a=ne_act//2,                     # Alpha orbitals
                                    num_elec_b=ne_act//2,                     # Beta orbitals
                                    job_id=job_id,                            # QPU bitstring Job ID
                                    client=client,                            # Diagonalization engine worker
                                    energy_tol=energy_tol,                    # SQD option
                                    occupancies_tol=occupancies_tol,          # SQD option
                                    max_iterations=max_iterations,            # SQD option
                                    num_batches=num_batches,                  # Eigenstate solver option
                                    samples_per_batch=samples_per_batch,      # Eigenstate solver option
                                    symmetrize_spin=symmetrize_spin,          # Eigenstate solver option
                                    carryover_threshold=carryover_threshold,  # Eigenstate solver option
                                    max_cycle=max_cycle,                      # Eigenstate solver option
                                    mem=mem,                                  # Memory per Worker (Gb)
                                    )

    # e   : SQD-based estimate of the energy
    # rdm1: Spin-summed 1-particle reduced density matrix
    e, rdm1 = result[0], result[1]
    rho_approximation = update_rdm(mc, rdm1).copy() # <--- Reconstruct the one-body density matrix in the atomic orbital basis to update the external potential due to the solvent

    if with_solvent_e is not None:
        # Subsequent update loops enter here
        edup = np.einsum('ij,ji->', with_solvent_v, rho_approximation) # <-- edup: Incrementing the energy calculation with subsequent iterations
        e += ecore + with_solvent_e - edup

    else:
        # First update loop enters here
        e += ecore # Pulled from the CAS space object (molecule's core energy)

    # Outputs:
    # with_solvent_e : scalar energy correction due to solvent polarization
    # with_solvent_v : Fock-like matrix to be added to the core Hamiltonian in SCF
    with_solvent_e, with_solvent_v = mc.with_solvent._get_vind(rho_approximation)
    data.append((iiter, float(e), job_id))
    print(f">>>>> END IITER {iiter}")
    print(f">>>>> TOTAL ENERGY: {e}\n")

In [ ]:
# If you make a mistake and need to cancel Serverless jobs
cancel = False
for job in client.jobs():
    print(f"ID {job.job_id}: {job.status()}")
    if cancel:
        job.cancel()

<div class="alert alert-block alert-danger">

### Report your plot

**SQD/IEF-PCM(cc-pVDZ) Energy Convergence Plot**: To assist you in visualizing your final results, we have provided a plotting tool. You must generate a plot for the successfully computed molecule $\textit{using the function provided}$. 

</div>

In [ ]:
# Plot your data
import matplotlib.pyplot as plt
from utils.visualization import plot_data

fig, ax = plot_data(data, baseline_1=hf_e, baseline_2=CASCI_E, name=molecule_name, save=True)
plt.show()

In [ ]:
# Verify your data
print(f"Molecule: {molecule_name}")
print(f"Hartree-Fock Energy: {hf_e}")
print(f"CASCI Energy: {CASCI_E}")
print(f"Hardware Data:\n{data}") # Make sure to include the job_ids' with this data (not doing so will result in disqualification as the winners reported data will be validated against the job_id data)

<div class="alert alert-block alert-danger">

### Record your SQD-based results

To report your results, use the function $report\_result()$. You can put your finalized classical energy and quantum energy values for each respective molecule. Make sure to put your answers into the `report_result` function, which will write them to a `results.json` file that will be used to compute your score. If you do not submit results from real quantum hardware, you will receive zero points for the section!

</div>

In [ ]:
from utils.helpers import report_result

# Overwrite current data in results.json (be careful, and save a backup file if uncertain)
report_result(molecule_name=molecule_name, hartree_fock_E=hf_e, casci_E=CASCI_E, sqd_E=data)

In [ ]:
grader_ch2_ex3(molecule_name, data)

<div class="alert alert-block alert-success">
Good job! You've finished Excercise 3!

Congratulations! You've finished all the problems!
</div>